In [6]:
import os
os.environ["WF_TRACE_SERVER_URL"] = "http://127.0.01:6345"

In [7]:
import weave

In [8]:
client = weave.init("remote_model_demo_2")

Logged in as Weights & Biases user: timssweeney.
View Weave data at https://wandb.ai/timssweeney/remote_model_demo/weave


In [9]:
# Demonstrates creating a model in python

from weave.builtin_objects.models.CompletionModel import LiteLLMCompletionModel

model = LiteLLMCompletionModel(
    model="gpt-4o",
    messages_template=[
        {"role": "system", "content": "Please extract the name and age from the following text"},
        {"role": "user", "content": "{user_input}"},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "Person",
            "schema": {
                "type": "object",
                "properties": {
                    "age": {"type": "integer"},
                    "name": {"type": "string"},
                },
            }
        }
    },
)

res = model.predict(user_input="Hello, my name is Fred and I am 30 years old.")

print(model.ref.uri())

print(res)

🍩 https://wandb.ai/timssweeney/remote_model_demo/r/call/0193b497-27e9-7f31-8f26-a24d76254d32


{'name': 'Fred', 'age': 30}

In [10]:
# Demonstrates calling a model created in python

from weave.trace_server.trace_server_interface import CallMethodReq

call_res = client.server.call_method(CallMethodReq(
    project_id=client._project_id(),
    object_ref=model.ref.uri(),
    method_name="predict",
    args={
        "user_input": "Hello, my name is Charles and I am 40 years old."
    }
))

call_res

CallMethodRes(call_id='0193b497-3c5b-7121-9a88-f14798eac933', output={'name': 'Charles', 'age': 40})

In [14]:
# Demonstrates creating a model in the UI - notice the digest match

from weave.trace_server.trace_server_interface import ObjCreateReq

obj_res = client.server.obj_create(ObjCreateReq.model_validate({'obj': {
    "project_id": client._project_id(),
    "object_id": "LiteLLMCompletionModel",
    "val": {
        "model": "gpt-4o",
        "messages_template": [{"role": "user", "content": "{user_input}"}],
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "Person",
                "schema": {
                    "type": "object",
                    "properties": {
                        "age": {"type": "integer"},
                        "name": {"type": "string"},
                    },
                },
            },
        },
    },
    "set_leaf_object_class": "LiteLLMCompletionModel",
}}))

obj_res

ObjCreateRes(digest='GSK0zJwYfB7Scv2StQWsxGtoxuvuGxTSEOFzKSrmkAg')

In [15]:
# Demonstrates fetching a model in python that was created in the UI

from weave.trace.refs import ObjectRef

gotten_model = weave.ref(ObjectRef(
    entity=client.entity,
    project=client.project,
    name="LiteLLMCompletionModel",
    _digest=obj_res.digest
).uri()).get()

res = gotten_model.predict(user_input="Hello, my name is Fred and I am 30 years old.")
res

🍩 https://wandb.ai/timssweeney/remote_model_demo/r/call/0193b498-a593-7391-b059-b32909b46f93


{'name': 'Fred', 'age': 30}